# https://www.yuque.com/ol1q37/gi94xp/uhqfah
# https://www.yuque.com/ol1q37/gi94xp/vy59wr

# 函数部分

In [12]:
def get_html(a):#解析网址顺便设置抽取语句
    from selenium import webdriver
    driver=webdriver.Chrome()
    driver.get(a)
    html = driver.page_source
    import re
    #global match
    match = re.search(r'<div class="main_27Lme">([\d\D]*?)</span></div></div>', html, re.MULTILINE)#要抽取的范围，正则匹配的内容需要在这范围里
    result = ""
    if match:
        result = match.group(0)
    html = result
    global reobj
    reobj = re.compile(r'<td>([\d\D]*?)</td><td><a href="([\d\D]*?)" class="">([\d\D]*?)</a></td><td>([\d\D]*?)</td><td>([\d\D]*?)</td><td>([\d\D]*?)</td><td>([\d\D]*?)</td></tr>', re.MULTILINE)
    return html
#以下为收集
def get_biaohao(html):
    for match in reobj.finditer(html):
        a=match.group(1)#标号
        list0.append(a)
    return list0
def get_wangzhi(html):
    for match in reobj.finditer(html):
        b=match.group(2)#url
        list1.append("https://pintia.cn"+b)
    return list1
def get_biaoti(html):
    for match in reobj.finditer(html):
        c=match.group(3)#标题
        list2.append(c)
    return list2
def get_fenshu(html):
    for match in reobj.finditer(html):
        d=match.group(4)#分数
        list3.append(d)
    return list3
def get_tongguoshu(html):
    for match in reobj.finditer(html):
        e=match.group(5)#通过数
        list4.append(e)
    return list4
def get_tijiaoshu(html):
    for match in reobj.finditer(html):
        f=match.group(6)#提交数
        list5.append(f)
    return list5
def get_tongguolv(html):
    for match in reobj.finditer(html):
        g=match.group(7)#通过率
        list6.append(g)
    return list6
def extract_info(html,t):#综合应用并保存入results
    global list0,list1,list2,list3,list4,list5,list6
    list0=[]
    list1=[]
    list2=[]
    list3=[]
    list4=[]
    list5=[]
    list6=[]
    biaohao = get_biaohao(html) # ['第6章函数-1', '第6章函数-2', '第6章函数-3', '第6章函数-4', '第6章函数-5', '第6章函数-6']
    biaoti = get_biaoti(html)
    fenshu = get_fenshu(html)
    tongguoshu = get_tongguoshu(html)
    tijiaoshu = get_tijiaoshu(html)
    tongguolv = get_tongguolv(html)
    wangzhi = get_wangzhi(html)
    types = [t[1]]*len(biaohao) # ['函数题', '函数题', '函数题', '函数题', '函数题', '函数题']
    results = []
    for a,b,c,d,e,f,g,h in zip(biaohao,biaoti,fenshu,tongguoshu,tijiaoshu,tongguolv,wangzhi,types):
        results.append((a,b,c,d,e,f,g,h))
    return results
def save2csv(results):#保存数据为csv文件
    import codecs
    f = codecs.open("D:\\ex5_new.csv","w","utf-8-sig")
    for result in results:
        f.write(",".join(result)+"\n")
    f.close()
def save_htmls(urls,filename):#分别保存为html
    for idx,url in enumerate(urls):
        driver.get(url)
        time.sleep(3)
        html = driver.page_source
        f = codecs.open(str(filename[idx]).replace('*','_')+".html","w","utf-8-sig")
        f.write(html)
        f.close()        

# part1  正则表达式循环抽取

In [13]:
urls = [("https://pintia.cn/problem-sets/1111652100718116864/problems/type/6","函数题"),
        ("https://pintia.cn/problem-sets/1111652100718116864/problems/type/7","编程题")]
#遍历循环这些链接，将结果存放在results中
results = []
for t in urls:
    html = get_html(t[0])
    li = extract_info(html,t)
    results.extend(li)
#将results中的数据持久化到csv文件中
save2csv(results)

# part2  保存为html

In [ ]:
save_htmls(list1,list2)

# part3 数据库

In [3]:
import sqlite3
import os

current_address = os.path.abspath('.')
db_address = os.path.join(current_address, "ceshi.db")
tbl_name = "shujubiao"
if os.path.exists(db_address):
    os.remove(db_address)



In [16]:
conn = sqlite3.connect(db_address)
cur = conn.cursor()
sql = 'CREATE TABLE "%s" (biaohao CHAR(25),biaoti CHAR(25),fenshu CHAR(25),tongguoshu CHAR(25),tijiaoshu CHAR(25),tongguolv CHAR(25),wangzhi CHAR(50),type CHAR(25))' % str(tbl_name)
cur.execute(sql)
conn.commit()
for i in results:
    sql = 'INSERT INTO shujubiao(biaohao,biaoti,fenshu,tongguoshu,tijiaoshu,tongguolv,wangzhi,type) VALUES("%s","%s","%s","%s","%s","%s","%s","%s")' % i
    cur.execute(sql)
conn.commit()
conn.close()